**Library import**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
user_agent = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

**Detiknews scraping**

In [ ]:
link = 'https://news.detik.com/berita/d-7768795/terjadi-bersamaan-rumah-di-cengkareng-kebakaran-dan-kebanjiran'
news = requests.get(link,user_agent).text
news_soup = BeautifulSoup(news, 'lxml')
date = news_soup.find('div', class_='detail__date').text
content_container = news_soup.find('div', class_='detail__body-text itp_bodycontent')
location = content_container.find('strong').text
contents = content_container.find_all('p')
remove_css = content_container.find_all('p', class_='para_caption')
contents = [x for x in contents if x not in remove_css]
text = []
for x in contents:
  y = x.text
  text.append(y)
  content = ' '.join(text).replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','').replace('SCROLL TO CONTINUE WITH CONTENT', '').replace('\n', '')

content = location + ' - ' + content
content

'Jakarta - Rumah di Cengkareng, Jakarta Barat (Jakbar), mengalami kebakaran dini hari tadi. Pada saat yang bersamaan, wilayah tersebut juga tergenang banjir. "Kebakaran melanda bangunan rendah," demikian keterangan Dinas Penanggulangan Kebakaran dan Penyelamatan (Gulkarmat) Jakarta, Sabtu (8/2/2025). Kebakaran itu terjadi sekitar pukul 03.00 WIB. Rumah yang terbakar berlokasi di Jalan Taman Palem Lestari, RT 07 RW 08, Kelurahan Cengkareng Barat, Kecamatan Cengkareng, Jakbar. Damkar mengerahkan 13 unit mobil pompa air dan 65 personel untuk memadamkan api. Proses pemadaman berlangsung cukup lama, yakni 2,5 jam. Operasi pemadaman dimulai pukul 03.01 WIB. Sampai pukul 04.40 WIB petugas pemadam kebakaran (damkar) melanjutkan dengan proses pendinginan. "Tahap pendinginan dilakukan untuk mengurai material yang mudah terbakar," katanya. Operasi pemadaman dinyatakan selesai pukul 05.31 WIB. Diduga kebakaran dipicu korsleting listrik. "Diduga (akibat) korsleting listrik. Kronologi: sekitar pukul

**Detiknews scraping**

In [ ]:
def detik_page(query, start_date, end_date):
  global user_agent
  url = f'https://www.detik.com/search/searchnews?query={query}&siteid=3&sortby=time&sorttime=1&fromdatex={start_date}&todatex={end_date}&result_type=latest'
  text = requests.get(url, user_agent).text
  sop = BeautifulSoup(text, 'lxml')
  try:
    paging = sop.find_all('div','pagination text-center mgt-16 mgb-48')[0].find_all('a')[-2]
    last_page = paging.text
  except:
    last_page = 1
  return last_page

def scrape_detik(query, start_date, end_date, file_name):
    global user_agent
    last_page = detik_page(query, start_date, end_date)
    data = []
    id = 0
    for page in range(1,int(last_page)+1):
        url = f'https://www.detik.com/search/searchnews?query={query}&siteid=3&sortby=time&sorttime=1&fromdatex={start_date}&todatex={end_date}&result_type=latest&page={page}'
        text = requests.get(url,user_agent).text
        soup = BeautifulSoup(text, 'lxml')
        articles_container = soup.find_all('article', class_='list-content__item')
        print(f'page: {page}')
        for article in articles_container:
            headline = article.find('div', 'media__text').find('a').text
            link = article.find('div', 'media__text').find('a')['href']
            try:
              news = requests.get(link,user_agent).text
              news_soup = BeautifulSoup(news, 'lxml')
              date = news_soup.find('div', class_='detail__date').text
              content_container = news_soup.find('div', class_='detail__body-text itp_bodycontent')
              location = content_container.find('strong').text
              contents = content_container.find_all('p')
              remove = content_container.find_all('p', class_='para_caption')
              contents = [x for x in contents if x not in remove_css]
              text = []
              id += 1
              # content = content_container.text.replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','') \
              # .replace('SCROLL TO CONTINUE WITH CONTENT', '').replace('\n\n\r\n\r\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\r\n\n','') \
              # .replace('\n\n\n','').replace('\n\n','')
              for x in contents:
                  y = x.text
                  text.append(y)
                  content = ' '.join(text).replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','').replace('SCROLL TO CONTINUE WITH CONTENT', '').replace('\n', '')

              content = location + ' - ' + content
              print(f'article: {id}')

              data.append({'id': id,
                          'source': 'Detik',
                          'title': headline,
                          'url': link,
                          'content': content,
                          'date': date})
            except:
              pass

    df = pd.DataFrame(data)
    df.to_csv(f'./drive/MyDrive/RISET/DATASET/{file_name}', index=False)

In [ ]:
scrape_detik('Kecelakaan Ciawi', '05/02/2025', '16/02/2025', 'kecelakaanciawi.csv')

**Detiknews indeks**

In [ ]:
def detik_indeks_page(date):
  url = f'https://news.detik.com/berita/indeks?page=1&date={date}'
  text = requests.get(url).text
  sop = BeautifulSoup(text, 'lxml')
  try:
    paging = sop.find_all('div','pagination text-center mgt-16 mgb-16')[0].find_all('a')[-2]
    last_page = paging.text
  except:
    last_page = 1
  return last_page

def detik_news_indeks(date):
    last_page = detik_indeks_page(date)
    data = []
    for page in range(1,int(last_page)+1):
        url = f'https://news.detik.com/berita/indeks?page={page}&date={date}'
        text = requests.get(url).text
        soup = BeautifulSoup(text, 'lxml')
        articles_container = soup.find_all('article', class_='list-content__item')
        for article in articles_container:
            headline = article.find('div', 'media__text').find('a').text
            link = article.find('div', 'media__text').find('a')['href']
            data.append({'source': 'Detiknews',
                         'title': headline,
                         'url': link,
                         'date': date})

    df = pd.DataFrame(data)
    df.to_csv(f'./drive/MyDrive/RISET/indeks.csv', index=False)

In [ ]:
detik_news_indeks('01/28/2025')